## Part h: Cross-validation and resampling techniques

In [29]:
# Import libraies and functions:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# import our own implementations
import importlib, OLS, prepare_data, polynomial_features,K_fold
importlib.reload(OLS)
importlib.reload(prepare_data)
importlib.reload(polynomial_features)

from prepare_data import prepare_data
from polynomial_features import polynomial_features
from OLS import OLS_parameters
from K_fold import k_fold_split
from sklearn.model_selection import KFold

# Styling function
from Set_latex_params import set_mpl_latex_style
#set_mpl_latex_style()

In [5]:
#Prepare data
x, y, _, _, _, _, y_noisy = prepare_data(n=100)


In [27]:
deg = 5

folds = k_fold_split(x, y, k=5, shuffle=True, random_state=6114)
mses = []

for i, (train_idx, test_idx) in enumerate(folds):
    x_train_f = x[train_idx]
    x_test_f = x[test_idx]

    y_train_f = y[train_idx]
    y_test_f = y[test_idx]

    y_offset = y_test_f.mean()

    X_train_f = polynomial_features(x_train_f,deg,intercept=False)
    X_test_f = polynomial_features(x_test_f,deg,intercept=False)

    ols_betas = OLS_parameters(X_train_f,y_train_f)
    pred_y_f = np.dot(X_test_f,ols_betas)+y_offset
    mse_f = mean_squared_error(pred_y_f, y_test_f)

    mses.append(mse_f)

print(f"Mean across folds {np.mean(mses)}")


Mean across folds 0.10698697393022787


### Sanity check with imported library
Since there is a randomness to what elements are put in train and test sets, we cannot expect the results to be the same. But we can expect them to interact similarly

In [ ]:
deg = 5

folds = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(kf.split(X)):

